In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [2]:
con = sqlite3.connect("../data/Checking_Logs.sqlite")

## Using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
  - time should have the values: after and before

  - avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward

  - only take into account the users that have observations before and after
  
  - we still are not using the lab ’project1’

In [6]:
query = """
    WITH user_times AS (
        SELECT
            uid,
            CASE 
                WHEN strftime('%s', first_commit_ts) < strftime('%s', first_view_ts) THEN 'before'
                ELSE 'after'
            END AS time,
            (d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0 AS diff
        FROM test t
        JOIN deadlines d ON t.labname = d.labs
        WHERE t.labname <> 'project1'
    ),
    filtered_users AS (
        SELECT uid
        FROM user_times
        GROUP BY uid
        HAVING COUNT(DISTINCT time) = 2
    )
    SELECT time, AVG(diff) AS avg_diff
    FROM user_times
    WHERE uid IN (SELECT uid FROM filtered_users)
    GROUP BY time
    ORDER BY time DESC
"""

test_results = pd.read_sql(query, con)
test_results

,time,avg_diff
0,before,61.156632
1,after,105.229241


In [5]:
query = """
    WITH user_times AS (
        SELECT
            uid,
            CASE 
                WHEN strftime('%s', first_commit_ts) < strftime('%s', first_view_ts) THEN 'before'
                ELSE 'after'
            END AS time,
            (d.deadlines - strftime('%s', c.first_commit_ts)) / 3600.0 AS diff
        FROM control c
        JOIN deadlines d ON c.labname = d.labs
        WHERE c.labname <> 'project1'
    ),
    filtered_users AS (
        SELECT uid
        FROM user_times
        GROUP BY uid
        HAVING COUNT(DISTINCT time) = 2
    )
    SELECT time, AVG(diff) AS avg_diff
    FROM user_times
    WHERE uid IN (SELECT uid FROM filtered_users)
    GROUP BY time
    ORDER BY time DESC
"""

control_results = pd.read_sql(query, con)
control_results

,time,avg_diff
0,before,99.901448
1,after,118.144571


## Close the connection

In [7]:
con.close()

## Have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

В обеих группах виден прирост времени работы, но в тестовой группе наблюдается прирост почти в `два` раза, тогда как в контрольной группе всего около `20%`. Тоесть введение Newsfeed оказало положительное влияние.